In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model

import sagemaker
import boto3, re
from sagemaker.session import Session
from sagemaker import get_execution_role
from sagemaker.tensorflow.model import TensorFlowModel
`
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
tf.__version__

In [ ]:
if tf.executing_eagerly():
    tf.compat.v1.disable_eager_execution()

In [ ]:
IMSIZE = 49

Classes = ['speed limit 20 (prohibitory)', 'speed limit 30 (prohibitory)', 'speed limit 50 (prohibitory)',
           'speed limit 60 (prohibitory)', 'speed limit 70 (prohibitory)', 'speed limit 80 (prohibitory)',
           'restriction ends 80 (other)', 'speed limit 100 (prohibitory)', 'speed limit 120 (prohibitory)',
           'no overtaking (prohibitory)', 'no overtaking (trucks) (prohibitory)', 'priority at next intersection (danger)',
           'priority road (other)', 'give way (other)', 'stop (other)', 'no traffic both ways (prohibitory)',
           'no trucks (prohibitory)', 'no entry (other)', 'danger (danger)', 'bend left (danger)', 'bend right (danger)', 
           'bend (danger)', 'uneven road (danger)', 'slippery road (danger)', 'road narrows (danger)', 'construction (danger)',
           'traffic signal (danger)', 'pedestrian crossing (danger)', 'school crossing (danger)', 'cycles crossing (danger)',
           'snow (danger)', 'animals (danger)', 'restriction ends (other)', 'go right (mandatory)', 'go left (mandatory)', 
           'go straight (mandatory)', 'go right or straight (mandatory)', 'go left or straight (mandatory)', 
           'keep right (mandatory)', 'keep left (mandatory)', 'roundabout (mandatory)', 
           'restriction ends (overtaking) (other)', 'restriction ends (overtaking (trucks)) (other)']

def readimg(pth):
    img = cv2.imread(pth)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMSIZE, IMSIZE), interpolation = cv2.INTER_AREA)
    plt.imshow(img)
    plt.show()
    img = img/255.0
    img = np.array(img).reshape(-1, IMSIZE, IMSIZE, 3)
    return img 

In [ ]:
deepNet = load_model('deepNet.46-0.0982.h5')   # update the model to be deployed.

In [ ]:
# Creating the model signature i.e. input and output size.

prediction_signature = tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(
    {"images": deepNet.input}, {"detections": deepNet.output})

In [ ]:
# Creating path to save the model.

export_path = os.path.join(
    tf.compat.as_bytes('deepNet'),
    tf.compat.as_bytes('1'))

In [ ]:
builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_path)

In [ ]:
keras_session = tf.compat.v1.keras.backend.get_session()

In [ ]:
# Saving the model in protobuffer format for deployment.

builder.add_meta_graph_and_variables(keras_session, [tf.compat.v1.saved_model.tag_constants.SERVING],
                                     signature_def_map={'serving_default': prediction_signature,})

builder.save()

In [ ]:
# Creating the zip file of the model saved in protobuff format to store in the storage bucket.

!tar -C "$PWD" -czf deepNet-01.tar.gz deepNet/

In [ ]:
sagemaker_session = Session()
model_data = sagemaker_session.upload_data(path='deepNet-01.tar.gz', key_prefix='model')

In [ ]:
sagemaker_bucket = sagemaker_session.default_bucket()
print("Bucket name is:", sagemaker_bucket)

In [ ]:
!touch train.py  # create an empty .py file to create an entry point.

In [ ]:
# Any operation in AWS is done be creating a role. This step involves obtaining the role and uplaoding the model details to
# sagemaker

role = get_execution_role()

from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = f's3://{sagemaker_bucket}/model/deepNet-01.tar.gz',   # location of zip file
                                  role = role, # execution role 
                                  framework_version = '2.4.1', # update appropriate tensorflow version
                                  entry_point = 'train.py')

In [ ]:
# Deploying the model to the endpoint

predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

In [ ]:
# Obtaining the endpoint to make the predictions

endpoint = predictor.endpoint
endpoint

In [ ]:
endpoint = ''  # upload the endpoint obtained in the previous step.

predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint, sagemaker_session)

data = readimg('')   # <-- Add the image to be predicted

def predicted_sign(input_data):
    return Classes[np.argmax(predictor.predict(input_data)['predictions'][0])]

alg_prediction = predicted_sign(data)
alg_prediction